In [39]:
import import_ipynb
import abuse_detecting_preprocessing as pp#text_preprocessing(x) ,text_preprocessing_can_eng_num(x)
import pandas as pd
import numpy as np
from tqdm import tqdm
import fasttext
from glob import glob
import os
import random

In [78]:
def erase_bot(data):

    # 첫 번째 열의 값이 '@'로 시작하는 행을 필터링하여 봇으로 간주하고 제거
    idxs = data[data.str.startswith('@') == True].index
    data.loc[idxs] = None
    bot_cnt = len(idxs)
    print('data.shape : ', data.shape, 'None 처리한 봇 수 : ', bot_cnt, '\n')
    return data
# erase_bot 함수 사용 예시
# data = erase_bot(data)


In [41]:
def null_finder(data): # 공백만 있는 메세지 제거
    print('공백 제거 전\n', data.isnull().value_counts())
    data = data.dropna()
    print('공백 제거 후 \n', data.isnull().value_counts())
    
    return data

In [42]:
def erase_emoticons(data):
    cnt = 0
    try:
        cnt = 0
        for i in range(len(data)):
            if ':' in data[i]:
                l = data[i].find(':')
                r = data[i].rfind(':')
                data[i] = data[i][:l] + data[i][r+1:]
                cnt += 1
    except Exception as e:
        print("An error occurred:", e)
    return data

In [43]:
def equalmatchfinder(routes):
    routes = sorted(routes)
    team_l = {'T1' : 0, 'GEN' : 0, 'HLE' : 0, 'KT' : 0, 'KDF' : 0,
              'DK' : 0, 'FOX' : 0, 'NS' : 0, 'DRX' : 0, 'BRO' : 0}
    idx = 0
    for route in routes:
        for key in team_l.keys():
            if route.find(key) != -1:
                team_l[key] += 1
        idx += 1
        if len(set(list(team_l.values()))) == 1: # set으로 바꾸면 중복이 사라지기에 길이가 1이라는 뜻은 경기수가 전부 같음을 의미.
            print(idx,'번째에서 모두 같은 값입니다.\n', team_l)
    
    # 1~10 : fasttext 
    # 11~15 : activation learning
    # 16~20 : analysis
    # 21~25 : labeling

In [44]:
def n_grams(sentences, n):
    n_grams_list = []

    for sentence in sentences:
        n_grams_sentence = []
        sentence_length = len(sentence)

        if sentence_length < n:
            continue

        for i in range(sentence_length - n + 1):
            n_gram = sentence[i:i+n]
            n_grams_sentence.append(n_gram)

        n_grams_list.append(n_grams_sentence)
    print(f' n = {n}인 ngram 생성.')
    return n_grams_list

In [45]:
def fasttext_model(x,n):

    with open('tokenized_data.txt', 'w') as out:
        for line in tqdm(x, unit=' line'):
            out.write(' '.join(line) + '\n')
            
    model = fasttext.train_unsupervised('tokenized_data.txt',
                                    model = 'skipgram', lr = 0.05, dim = 100,
                                    ws = 5, epoch = 30, minn = 1, word_ngrams = n)
    return model

In [46]:
def making_fasttext_model(messages, N = 5):
    messages = erase_bot(messages)

    messages = null_finder(messages)
    
    messages = erase_emoticons(messages)
    
    words = pp.text_preprocessing_can_eng_num(messages)
        # @로 시작하는 채팅들을 Null로 설정 (봇 제거)
        # Null들을 제거
        # :로 시작하는 이모티콘들 제거
        # 메세지에서 한국어와 영어만 남긴다.
        # 텍스트에서 겹친 글자를 줄인다. (ㅋㅋㅋㅋㅋ > ㅋ)
        # 텍스트를 초성으로 바꾼다.
        # 바꾼 텍스트에서 겹모음을 바꾼다.
    bow = n_grams(words , N) # 리스트로 받아 각 문장을 5개씩 쪼갠다.
    model = fasttext_model(bow, N)
    return model


In [53]:
route = '/Users/kim-yongjun/Documents/chatting_data/LCK_spring_chat/*.csv'
routes = glob(route)
routes = sorted(routes)


In [81]:
equalmatchfinder(routes)
len(routes)
# 팀 간의 균일한 경기 수를 모아 fasttext 임베딩 모델을 구축

5 번째에서 모두 같은 값입니다.
 {'T1': 2, 'GEN': 2, 'HLE': 2, 'KT': 2, 'KDF': 2, 'DK': 2, 'FOX': 2, 'NS': 2, 'DRX': 2, 'BRO': 2}
10 번째에서 모두 같은 값입니다.
 {'T1': 4, 'GEN': 4, 'HLE': 4, 'KT': 4, 'KDF': 4, 'DK': 4, 'FOX': 4, 'NS': 4, 'DRX': 4, 'BRO': 4}
15 번째에서 모두 같은 값입니다.
 {'T1': 6, 'GEN': 6, 'HLE': 6, 'KT': 6, 'KDF': 6, 'DK': 6, 'FOX': 6, 'NS': 6, 'DRX': 6, 'BRO': 6}
20 번째에서 모두 같은 값입니다.
 {'T1': 8, 'GEN': 8, 'HLE': 8, 'KT': 8, 'KDF': 8, 'DK': 8, 'FOX': 8, 'NS': 8, 'DRX': 8, 'BRO': 8}
26 번째에서 모두 같은 값입니다.
 {'T1': 10, 'GEN': 10, 'HLE': 10, 'KT': 10, 'KDF': 10, 'DK': 10, 'FOX': 10, 'NS': 10, 'DRX': 10, 'BRO': 10}


55

In [60]:
import gc
Games = 26

df = pd.DataFrame()
for r in routes[:Games]:
    data = pd.read_csv(r)
    df = pd.concat([df, data])
    del data
    gc.collect()
df = df[['message']].reset_index(drop = True)
print(df.shape)
df.sample(5)


(482915, 1)


,message
7364,농심vsDRX는 슈카월드 선에서 정리되네
360461,혜지누님 미소 볼때마다 저도 기분이 좋아요 기다렸어요
254855,진지하게 DRX가 광동 잡을만함?
239958,와 ㅈㄴ꿀잼이다
36334,캬


In [79]:
model = making_fasttext_model(df['message'], 5)

data.shape :  (482915,) None 처리한 봇 수 :  4054 

공백 제거 전
 message
False    478850
True       4065
Name: count, dtype: int64
공백 제거 후 
 message
False    478850
Name: count, dtype: int64
An error occurred: 63670
 n = 5인 ngram 생성.


100%|████████████████████████████| 429823/429823 [00:00<00:00, 842357.43 line/s]
Read 11M words
Number of words:  115897
Number of labels: 0
Progress: 100.0% words/sec/thread:   51258 lr:  0.000000 avg.loss:  0.336031 ETA:   0h 0m 0s54843 lr:  0.040506 avg.loss:  0.565600 ETA:   0h12m 1s 3m14s


In [82]:
model.get_nearest_neighbors('ㅍㅔ-ㅇㅣ')

[(0.8992390036582947, 'ㅇㅣ-ㅋㅓ'),
 (0.8798893094062805, 'ㅣ-ㅋㅓ-'),
 (0.8755123019218445, '-ㅍㅔ-ㅇ'),
 (0.856319785118103, 'ㅍㅔ-ㅋㅓ'),
 (0.8452881574630737, '-ㅇㅣ-ㅋ'),
 (0.8185652494430542, '7ㅍㅔ-ㅇ'),
 (0.8144190907478333, 'gㅍㅔ-ㅇ'),
 (0.8141342997550964, 'ㄴㅍㅔ-ㅇ'),
 (0.8120465874671936, 'Gㅍㅔ-ㅇ'),
 (0.7978992462158203, '0ㅍㅔ-ㅇ')]

In [83]:
model.save_model('fasttext_model.bin')